### Условие задачи
**Дано:**

**7 файлов** `test1, ..., test7`:
- файлы содержат целые числа в текстовом формате
- первая строка - количество вершин $n$ в дереве фенотипа человека
- вторая строка содержит $n - 1$ целое число - номера родительских вершин (предков) в дереве для вершин $2, 3, ..., n$
- третья строка содержит $n$ целых чисел - количество информации во всех вершинах $IC(v)$, $v = 1, 2, ..., n$
- четвертая строка - количество болезней $m$
- следующие $m$ строк содержат описание болезней, в каждой строке по одной болезни:
    - болезнь $d_j$ описывается конечным множеством вершин $D_j$ в дереве фенотипа
    - первое число - количество элементов в $D_j$
    - остальные числа в строке - элементы множества $D_j$
- следующая строка - количество пациентов $nq$
- следующие $nq$ строк содержат описание пациентов, в каждой строке по одному пациенту:
    - пациент $p_i$ описывается конечным множеством вершин $Q_i$ в дереве фенотипа
    - первое число - количество элементов в $Q_i$
    - остальные числа в строке - элементы множества $Q_i$


**Требуется:**

Для входных файлов `test1, ..., test7` выполнить следующее.

Для каждого пациента $p_i$ найти номер болезни $\hat{d}_i$ согласно следующей формуле:

$\hat{d}_i = argmax_j\left(\sum\limits_{q \in Q_i}\left(\max\limits_{d \in D_j} IC(LCA(q, d))\right)\right)$, где

$LCA(u, v)$ - наименьший общий предок для вершин $v$ и $u$.

Номера $\hat{d}_i$ записать в строки файлов `result1, ..., result7` (в каждой строке - одно число).


**Для решения задачи понадобится:**

- загрузка данных
- сведение задачи поиска наименьшего общего предка к задаче поиска минимума в заданном диапазоне элементов массива
- обход дерева фенотипа в глубину с бэктрекингом (без использования рекурсии)
- реализовать алгоритм построения разреженной таблицы за время $O(n \cdot log(n))$
- реализовать алгоритм вычисления минимума в диапазоне `range minimum query`, использующий построенную таблицу, со сложностью $O(1)$
- использовать `taichi` или `numba` для компиляции и распараллеливания вычислений
- выполнить вычисления для всех тестов
- верифицировать результаты на соответствующих страницах контеста
- задокументировать все функции


**Материалы:**

- [Range Minimum Query](https://en.wikipedia.org/wiki/Range_minimum_query) (eng.)
- [LCA](https://cp-algorithms.com/graph/lca.html) (eng.)
- [Sparse Table](https://cp-algorithms.com/data_structures/sparse-table.html) (eng.)

In [1]:
import numpy as np
import networkx as nx
import itertools
import time
import os
from numba import njit, jit, prange, types
from numba.typed import List, Dict

In [2]:
def read_data(filename):
    """
    Reads the data of defined by the task statement format.
    
    Parameters
    ----------
    filename: str.
    A name of file to read from.
    
    Returns
    -------
    phenotype_vertices: int.
    Number of vertices in a tree.
    
    tree_data: np.array.
    The parent identifiers for vertices in a tree.
    
    vertices_information: np.array.
    Information content values of the corresponding vertices.
    
    diseases: numba.typed.List.
    Descriptions of diseases. Firstly comes the number of vertices in a tree that describe a disease,
    then the identifiers of vertices describing this disease.
    
    patients: numba.typed.List.
    Descriptions of patients. Firstly comes the number of vertices in a tree that describe a patient,
    then the identifiers of vertices describing this patient.
    
    number_diseases: int.
    Number of diseases. 
    """
    with open('/home/artem/Programming Python/diagnosis_problem_dir/' + filename, 'rt') as fuck:
        lines = fuck.readlines()
        
    phenotype_vertices = int(lines[0]) 
    number_diseases = int(lines[3]) 
    
    tree_data = np.array(List(map(int, lines[1].split()))) - 1
    vertices_information = np.array(List(map(int, lines[2].split())))
    
    diseases = make_list(lines[4:number_diseases+4])
    patients = make_list(lines[4+number_diseases+1:])
    
    return phenotype_vertices, tree_data, vertices_information, diseases, patients, number_diseases

In [3]:
@jit(forceobj=True)
def make_list(lines):
    """
    A function to create a list by using mapping a number in string format to int.
    
    Parameters
    ----------
    lines: str.
    A bunch of strings describing something.
    
    Returns
    -------
    : numba.typed.List
    An array of transformed to numbers strings.
    """
    arr = list()
    for line in lines:
        if line:
            arr.append(np.array(list(map(int, line.split()[1:]))) - 1)
    return List(arr)

In [4]:
@jit(parallel=True, forceobj=True)
def descendants(tree):
    """
    A function to identify children of vertices, if there are any.
    
    Parameters
    ----------
    tree: np.array.
    An array describing tree formation.
    
    Returns
    -------
    vertice_ancestor: dict.
    A dictionary, where a key is a vertice and values are children of this vertice.
    """
    # vertice_ancestor = Dict.empty(key_type=np.int64, value_type=int)
    vertice_ancestor = dict()
    for anc, vert in enumerate(tree, start=1):
        vertice_ancestor.setdefault(vert, []).append(anc)
    return vertice_ancestor

In [5]:
@jit(forceobj=True)
def EulerTour(root, phenotype_vertices):
    """
    A function to create eulerian tree traversal path. Eulerian path describes the order in which tree node was reached.
    Also gives information on which step got to vertexes.
    
    Parameters
    ----------
    root: int.
    An identifier of root of the tree to start from.
    
    phenotype_vertices: int.
    Number of vertices in a tree.
    
    Returns
    -------
    : numba.typed.List.
    Euler tour. List contains of information content values about each node, not a node itself.
    
    : numba.typed.List.
    An array depicting a step when a node was visited. Indexes of this array depict a node number.
    """
    visited = np.zeros(phenotype_vertices)
    first_met = np.zeros(phenotype_vertices, dtype=int)
    euler_tour = List()  # Для бэктрекинга
    stack = [root]
    while stack:
        node = stack.pop(-1) # Вынимаем последний элемент
        if not visited[node]:
            visited[node] = True
            first_met[node] = len(euler_tour)
            euler_tour.append(vertices_information[node])
            # euler_tour.append(node)
            for p in D.get(node, [])[::-1]:  # Для обхода с левой ветки
                if not visited[p]:
                    stack.append(node)  # Ещё раз кладём тот узел, куда нужно вернуться
                    stack.append(p)
                    
        euler_tour.append(vertices_information[node])
        # euler_tour.append(node)
    return List(euler_tour), List(first_met)

In [6]:
@njit(parallel=True)
def buildSparse(input_array):
    """
    A function that creates a sparce table: a table containing minimums of each segment, whose lengths are equal to powers of two.
    
    Parameters
    ----------
    input_array: numba.typed.List.
    An array with data on which the matrix is built.
    
    Returns
    -------
    SparceTable: np.array.
    Created and filled sparce table.
    """
    elements_number = len(input_array)
    log = int(np.ceil(np.log2(elements_number)))
    
    SparceTable = np.zeros((elements_number, log), dtype = np.int32)
    for row in prange(elements_number):
        SparceTable[row][0] = input_array[row]
        
    for j in range(1, log): 
        for i in prange(elements_number - (1 << j) + 1):
        # while (int(i + (1 << j) - 1)) < elements_number:  # counting on intervals of len 2^power
                SparceTable[i][j] = min(SparceTable[i][j-1], SparceTable[i + (1 << (j-1))][j-1])

    return SparceTable

In [7]:
@njit
def RMQ(ST, left, right):
    """
    A function which does Range Minimum Query request -- request of a minimum on a segment in the array.
    
    Parameters
    ----------
    ST: np.array.
    Sparce table to use in RMQ task. Allows to find a minimum much faster.
    
    left: int.
    Left border of a segment to look for a minimum in.
    
    right: int.
    Right border of a segment to look for a minimum in.
    
    Returns
    -------
    : int.
    A requested minimum.
    """
    if left > right:
        left, right = right, left
        
    k = int(np.floor(np.log2(right - left + 1)))
    return min(ST[left][k], ST[right - (1 << k) + 1][k])

In [8]:
@njit(parallel=True)
def disease_determination(tour, patients, number_diseases, diseases, fm):
    """
    A function to find a disease for a patient. It is a programmed formula from the statement of the problem.
    
    Parameters
    ----------
    tour: numba.typed.List.
    Euler tour.
    
    patients: numba.typed.List.
    Descriptions of patients.
    
    number_diseases: int.
    Number of diseases.
    
    diseases: numba.typed.List.
    Descriptions of diseases.    
    
    fm: numba.typed.List.
    An array depicting a step when a node was visited. Indexes of this array depict a node number.
    
    Returns
    -------
    answer: numba.typed.List.
    The obtained result of formula's solvation. Created just to use it later as an answer-file filler.
    """
    ST = buildSparse(tour)
    answer = List()
    for patient in patients:
        information_sum = np.zeros(number_diseases)
        for p in patient: 
            for index, disease in enumerate(diseases):
                maximum = 0
                for d in disease:
                    ancestor = RMQ(ST, fm[d], fm[p])
                    if ancestor > maximum: 
                        maximum = ancestor
                information_sum[index] += maximum
        # os.system(f'echo {np.argmax(information_sum) + 1} >> {filename}_result.txt') # Нумба не может в файлы
        answer.append(np.argmax(information_sum) + 1)
    return answer

In [9]:
"""
A cell that calls all the functions necessary to solve the problem and writes the received response to a file in the required format.
"""
for filename in ['test1', 'test2', 'test3', 'test4', 'test5', 'test6', 'test7']:
    !rm {filename}_result.txt
    time_start = time.time()
    phenotype_vertices, tree, vertices_information, diseases, patients, number_diseases = read_data(filename)
    D = descendants(tree)
    tour, fm = EulerTour(0, phenotype_vertices)
    ans = disease_determination(tour, patients, number_diseases, diseases, fm)

    for element in ans:
        os.system(f'echo {element} >> {filename}_result.txt')

    print(f'{filename} is done in {time.time() - time_start} seconds')

test1 is done in 8.583899021148682 seconds
test2 is done in 0.5470180511474609 seconds
test3 is done in 64.35630679130554 seconds
test4 is done in 372.4092664718628 seconds
test5 is done in 3992.378957271576 seconds
test6 is done in 4628.8190767765045 seconds
test7 is done in 11605.575488567352 seconds
